# Save user private key in GCP KMS

#### Step 1. Activate Cloud Key Management Service (KMS) API
https://console.cloud.google.com/marketplace/product/google/cloudkms.googleapis.com

#### Step 2. Create a Google service account that has the role of Cloud KMS Admin
https://console.cloud.google.com/iam-admin/serviceaccounts

Generate and download a private key for the service account in JSON format

#### Step 2. Install KMS client lib

pip install google-cloud-kms

#### Step 3. Connect to KMS

In [2]:
from google.cloud import kms
from google.oauth2 import service_account

In [34]:
# SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'
SERVICE_ACCOUNT_FILE = '../kms-admin.json'

project_id = 'wallet-login-45c1c'

In [36]:

# Create the credentials object from the service account file.
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

# Create the client with the credentials.
client = kms.KeyManagementServiceClient(credentials=credentials)


create a key ring, a key, and a key version. You can specify the name of the key when you create it. 
For example, you can use the following code to create a symmetric encryption key named my-key in a key ring named my-key-ring in the global location

In [ ]:


# Build the parent name from the project and location.
parent = f"projects/{project_id}/locations/global"

# Build the key ring name.
key_ring_name = f"{parent}/keyRings/my-key-ring"

# Build the key name.
key_name = f"{key_ring_name}/cryptoKeys/my-key"

# Create the key ring.
key_ring = {}
client.create_key_ring(request={"parent": parent, "key_ring_id": "my-key-ring", "key_ring": key_ring})

# Create the key.
purpose = kms.CryptoKey.CryptoKeyPurpose.ENCRYPT_DECRYPT
crypto_key = {"purpose": purpose}
client.create_crypto_key(request={"parent": key_ring_name, "crypto_key_id": "my-key", "crypto_key": crypto_key})


You can use the client library to encrypt and decrypt data with the key you created. For example, you can use the following code to encrypt and decrypt a message using my-key

In [39]:

# Build the key name.
key_name = f"projects/{project_id}/locations/global/keyRings/my-key-ring/cryptoKeys/my-key"

# The message to encrypt.
message = b"Hello Gladius!"

# Encrypt the message.
response = client.encrypt(request={"name": key_name, "plaintext": message})
ciphertext = response.ciphertext

print(ciphertext)

b'\n$\x00\x17ax<;\xdaDL\x10\xf6y\xcfu\xdd\xa3 \x85\xda1iS\xfdVv\xbb\x81~\xfa\x02\xc9\x9903\x1bW\x127\x00T\xe6\xeb<|\x17\xdb+\xa4\n\x0fU\x12\x1f\xee\x7f\xa3,s\xc8A\x90\xb9$\x7f\x83p\xf0\xd3\xcct\xdd\x91\xe8\x94\x9a@\x8b\xd3\xc8\x8f\x0fP(\x13\x8b\x12x\xb0-\xd2\x9e\xc3\x90'


In [38]:

# Decrypt the message.
response = client.decrypt(request={"name": key_name, "ciphertext": ciphertext})
plaintext = response.plaintext

print(plaintext)

# Verify that the decrypted message matches the original message.
# assert plaintext == message

b'Hello Gladius!'


Use the Firebase SDK for Cloud Functions to create a function that uses the encrypted user private key.
https://stackoverflow.com/questions/63248432/how-to-encrypt-and-securely-store-and-transmit-private-keys-of-ssh
You can use any trigger type that suits your use case, such as HTTP, Firestore, Storage, etc. For example, you can use the following code to create a Firestore trigger that encrypts the user private key when a new user document is created

In [ ]:
# NOT WORKING

from firebase_admin import firestore
from firebase_functions import event_handler


# Initialize Firebase app with admin privileges.

if not firebase_admin._apps:
    cred = credentials.Certificate("../firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)
# Get a reference to the firestore database
db = firestore.client()


app = firestore.initialize_app()

# Get a reference to the Firestore service.
db = firestore.client()

# Define a function that encrypts the user private key when a new user document is created in Firestore.
@event_handler.on_document_created(reference="/users/{user_id}")

def encrypt_user_private_key(event):
  # Get the user ID from the event parameters.
  user_id = event.params.get("user_id")

  # Get the user private key from the event data.
  user_private_key = event.data.data().get("private_key")

  # Encrypt the user private key with Google KMS.
  response = client.encrypt(request={"name": key_name, "plaintext": user_private_key})
  encrypted_user_private_key = response.ciphertext

  # Update the user document with the encrypted user private key.
  db.collection("users").document(user_id).update({"private_key": encrypted_user_private_key})


### firebase save user private key in GCP KMS on create user event

In [29]:
import firebase_admin
from firebase_admin import auth, credentials, firestore

#from google.cloud import kms_v1
from google.cloud import kms
from google.oauth2 import service_account


In [ ]:
# SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'
SERVICE_ACCOUNT_FILE = '../kms-admin.json'

project_id = 'wallet-login-45c1c'

In [30]:
if not firebase_admin._apps:
    cred = credentials.Certificate("../firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)
# Get a reference to the firestore database
db = firestore.client()

In [32]:

def generate_private_key():
    # Generate or retrieve the private key for the user
    # You can implement your logic here to generate or retrieve the private key
    private_key = "USER_PRIVATE_KEY"
    return private_key


# Create a Firebase Auth user creation event handler.

 
def create_user(event):
    # Get the user's UID from the event data
    uid = event.uid

    # Generate or retrieve the user's private key
    private_key = generate_private_key()

    # Encrypt the private key using Google KMS
    encrypted_private_key = encrypt_private_key(private_key)

    # Save the encrypted private key in Firestore
    save_encrypted_private_key(uid, encrypted_private_key)


# Implement the encrypt_private_key() function to encrypt the private key using Google KMS.


def encrypt_private_key(private_key):

    # Build the key name.
    key_name = f"projects/{project_id}/locations/global/keyRings/my-key-ring/cryptoKeys/my-key"


    # Convert the private key to bytes
    private_key_bytes = private_key.encode('utf-8')

    # Prepare the request
    request = {
        'name': key_name,
        'plaintext': private_key_bytes,
    }

    # Call the encrypt API
    response = client.encrypt(request)

    # Retrieve the encrypted ciphertext
    encrypted_private_key = response.ciphertext
    return encrypted_private_key



def save_encrypted_private_key(uid, encrypted_private_key):
    # Define the document reference for the user
    doc_ref = db.collection('users').document(uid)

    # Update the document with the encrypted private key
    doc_ref.update({'encrypted_private_key': encrypted_private_key})

